In [ ]:
!pip install pyathena
from athena_querying import AthenaQuery
from athena_common_queries import *

In [ ]:
try:
    import psycopg2
except:
    print("Failed ot import psychopg2, trying to install it")
    !{sys.executable} -m pip install psycopg2-binary
    import psycopg2
    print("Successfully installed")
    
    
try:
    import dateparser
except:
    print("Failed ot import dateparser, trying to install it")
    !{sys.executable} -m pip install dateparser
    import dateparser
    print("Successfully installed")
    
import data_pier_querying

In [ ]:
from datetime import datetime, timedelta, date

# Settings

In [ ]:
num_days_to_query = 1
#from_datetime = datetime.now() - timedelta(days = 5)
from_datetime = datetime(year=2020, month=1, day=18)
to_datetime = from_datetime+ timedelta(days=num_days_to_query)

# Athena (Kinesis) Data

In [ ]:
aq = AthenaQuery()

In [ ]:
aq.connect()

In [ ]:
athena_database = "ms_data_lake_production"
athena_raw_events_table = "ms_data_stream_production_processed"

In [450]:
#query = "select context.page_url, body.event_name, count(*) from "+athena_database+"."+athena_raw_events_table
#query += " where partition_0='2019' and partition_1>='12' and partition_2>='05' group by 1,2"

In [ ]:
query = create_generic_event_query(from_datetime, to_datetime, include_device_type_data=True)

full_query = "select * from (%s) where country_code ='sg'" %query

In [ ]:
print(full_query)

In [ ]:
athena_full_events_df = aq.query(query)

In [ ]:
athena_full_events_df.head(20)

In [ ]:
#-------------------------------------------------

In [ ]:
df["sent_at_date"] = df.apply (lambda row: row.sent_at_timestamp[:10], axis=1)

In [ ]:
df.head()

In [ ]:
df[(df.is_blog==False) & (df.is_unbounce == False) &  (df.is_embed == False)].groupby(['sent_at_date', 'country_code', 'is_test']).count()

In [ ]:
df.head()

In [ ]:
sg_shop_pageviews = df[(df.is_blog==False) & (df.is_unbounce == False) &  (df.is_embed == False) & (df.country_code=='sg')]

In [ ]:
sg_shop_pageviews.head(5)

In [ ]:
sg_shop_pageviews.slug.head()

In [ ]:
a = sg_shop_pageviews.head(100).apply(lambda row: row.event_name) #str(row.slug).split("/")[0] if row.slug else "/", axis=1) # some weird shit of it being a number sometimes.

In [ ]:
a.head(30)

In [ ]:
sg_shop_summary = sg_shop_pageviews.groupby(["sent_at_date" , "country_code", "section"])

In [ ]:
sg_shop_summary

# Getting the Segment Events

In [ ]:
#from importlib import reload
reload(data_pier_querying)

In [ ]:
dp_querying = data_pier_querying.DataPierQuerying()
dp_querying.connect()

In [ ]:
tables_df = dp_querying.query_to_dataframe("select * from information_schema.tables")

In [ ]:
segment_event_tables_df = tables_df[tables_df.table_schema=="moneysmartsg_prod"]["table_name"]


In [ ]:
segment_event_tables_df[segment_event_tables_df.str.contains('shar')]

In [ ]:
# These are taken from the dictionary in https://docs.google.com/spreadsheets/d/1HICh77BoGMIat9K3NPwz3pBayJWiAr0ohAlTuv7dr80/edit#gid=1882048411
expected_events_str = """
LeadGeneration.ClickConversion
LeadGeneration.FormStepCompleted
LeadGeneration.FormSubmitted
LeadGeneration.PaymentCompleted
LeadGeneration.ThankYou
LeadGeneration.RedirectCompleted
UserEngagement.ShowedMoreDetails
UserEngagement.ViewedMoreDetails
UserEngagement.SortedList
UserEngagement.UsedHelpHints
UserEngagement.ClickedMenuItem
UserEngagement.QuestionAnswered
UserEngagement.ShowMoreFilter
UserEngagement.ShowMoreOptions
UserEngagement.ClickedFilter
UserEngagement.ButtonClick
UserAuth.LoggedIn
UserAuth.RegisteredAccount
UserAuth.LoggedOut
UserFeedback.ModalDisplayed
UserFeedback.MoodSubmitted
UserFeedback.FeedbackSubmitted
UserFeedback.MoreFeedback
ABTest.Conversion
UserView.WidgetLoad
EmailCapture
PageView
Sharing
Reading
NewsLetterPopup
"""
expected_events = [z.strip() for z in expected_events_str.split("\n") if len(z.strip())>0]

In [ ]:
expected_events_and_segment_tables = []
special_maps = {
    "PageView": "pages"
}
for event in expected_events:
    if event in special_maps:
        new_event_name = special_maps[event]
    else:
        new_event_name = ""
        for i, c in enumerate(event):
            if i==0:new_event_name+=c.lower()
            elif str.isupper(c): 
                if i>0 and event[i-1]!=".":
                    new_event_name += "_"
                new_event_name += c.lower()
            elif c==".": new_event_name += "_"
            else: new_event_name+= c
    expected_events_and_segment_tables.append([event, new_event_name])

In [ ]:
expected_events_and_segment_tables

## Check for missing tables

Expect some random events not to be in Segment, or blog specific ones that haven't been deployed to SG and HK

In [ ]:
# Check all the event tables exist
expected_event_segment_tables = [z[1] for z in expected_events_and_segment_tables]
segment_table_names = segment_event_tables_df.to_list()
missing_event_tables = [z for z in expected_event_segment_tables if z not in segment_table_names]
missing_event_tables

In [ ]:
expected_events_and_segment_tables

In [ ]:
# Removing the missing ones from the query list
events_and_tables_to_get_from_data_pier = [z for z in expected_events_and_segment_tables if z[1] not in missing_event_tables]

# Removing a problematic one (doesn't have context_page_url in it, and very unimportant
events_and_tables_to_get_from_data_pier = [z for z in events_and_tables_to_get_from_data_pier if z[1] not in ["user_auth_logged_out",]]

In [ ]:
len(events_and_tables_to_get_from_data_pier)

In [ ]:
cols = dp_querying.query_to_dataframe("""
select column_name, data_type, count(*) from information_schema.columns 
where 
table_name in  ('"""+"','".join([z[1] for z in events_and_tables_to_get_from_data_pier])+"""')
and table_schema='moneysmartsg_prod'

group by 1,2
""")

In [ ]:
cols.sort_values(["count"])

In [ ]:
cols[cols.column_name=="referrer"]

In [ ]:
segment_columns_to_query = [
    # "sent_at", - don't use this, use timestamp
    "timestamp",
    #"event", - going to get that implied from the table.
    # "status", # TODO: would like to have this, but not sure which column, or which tables.  Maybe just not used much, so only do for the 4 tables.
    "anonymous_id",
    "context_page_url",
    # "referrer", #maybe only used in pages table??
    "context_ip", 
    "context_user_agent"]

In [ ]:
tables_without_right_columns = dp_querying.query_to_dataframe("""
    select table_name, column_name, count(*) from information_schema.columns 
    where 
    table_name in  ('"""+"','".join([z[1] for z in events_and_tables_to_get_from_data_pier])+"""')
    and table_schema='moneysmartsg_prod'
    and column_name not in  ('"""+"','".join(segment_columns_to_query)+"""')
    group by 1,2
    """)
#tables_without_right_columns
print("this doesn't work yet.")

In [ ]:
segment_date_constraint = " timestamp >= '%s' and timestamp < '%s' " % (from_datetime.isoformat(), to_datetime.isoformat())

In [ ]:
query = "select {cols} from moneysmartsg_prod.{table} where {date_constraint}".format(cols=", ".join(segment_columns_to_query), 
                                                                   table="pages",
                                                                   date_constraint =segment_date_constraint)

In [ ]:
query

In [ ]:
pages = dp_querying.query_to_dataframe("select * from  moneysmartsg_prod.pages where  timestamp > '2020-01-08T15:24:00.263926' and timestamp < '2020-01-15T15:24:00.263966' limit 1000")

In [ ]:
click_conversions = dp_querying.query_to_dataframe("select * from  moneysmartsg_prod.lead_generation_click_conversion where  timestamp > '2020-01-08T15:24:00.263926' and timestamp < '2020-01-15T15:24:00.263966' limit 1000")

In [ ]:
click_conversions.head()

In [ ]:

for a in click_conversions.keys():
    print(a)

In [ ]:
dp_querying.query_to_dataframe("""SELECT
    nmsp_parent.nspname AS parent_schema,
    parent.relname      AS parent,
    nmsp_child.nspname  AS child_schema,
    child.relname       AS child
FROM pg_inherits
    JOIN pg_class parent            ON pg_inherits.inhparent = parent.oid
    JOIN pg_class child             ON pg_inherits.inhrelid   = child.oid
    JOIN pg_namespace nmsp_parent   ON nmsp_parent.oid  = parent.relnamespace
    JOIN pg_namespace nmsp_child    ON nmsp_child.oid   = child.relnamespace
WHERE parent.relname='%s';""")%"pages"

In [ ]:
pd.get_option("display.max_colwidth", 200)
indexes = dp_querying.query_to_dataframe("""SELECT
    indexname,
    indexdef
FROM
    pg_indexes
WHERE
    tablename = '%s';""" % "pages")

for a in indexes.values:
    print(a)

In [ ]:
# The meat of it
start_time = datetime.now()
event_name_to_rows = {}
for i, (event_name, table_name) in enumerate(events_and_tables_to_get_from_data_pier):
    table_start_time = datetime.now()
    print("querying table %s / %s (%i/%i)" % (table_name, event_name, i+1, len(events_and_tables_to_get_from_data_pier)))
    query = "select {cols} from moneysmartsg_prod.{table} where {date_constraint}".format(cols=", ".join(segment_columns_to_query), 
                                                                   table=table_name,
                                                                   date_constraint =segment_date_constraint)
    
    events = dp_querying.query_to_dataframe(query)
    events["event_name"] = event_name #fills the entire column with the same value
    print("Got %i events"% len(events))
    event_name_to_rows[event_name]=events
    
    table_download_time = (datetime.now()-table_start_time).total_seconds()
    time_since_start = (datetime.now()-start_time).total_seconds()
    print("It took %.1f seconds to download from the table (%.1f seconds overall)" %(table_download_time, time_since_start))
    print()
    # if i>4:break

In [408]:
# Merge tables
segment_combined_df = pd.DataFrame()
#combined_df = pd.DataFrame(columns=event_name_to_rows["LeadGeneration.ClickConversion"].columns)
"""for event_name, event_df in event_name_to_rows.items():
    print(len(event_df))
    combined_df.append(event_df, ignore_index=True)
    print(len(combined_df))
#combined_df.astype({"event_name":"category"})
"""

segment_combined_df = combined_df.append(list(event_name_to_rows.values()))

In [ ]:
segment_combined_df.head()

In [ ]:
segment_combined_df.groupby(["event_name"]).count()

In [ ]:
segment_combined_df.rename(columns={"context_page_url":"page_url"}, inplace=True)
segment_combined_df.head(5)

# Merging Segment and Kinesis Events

In [ ]:
# Make names clear e.g. s_...

# Check the timezone / timestamps match
# Athena raw stuff is in UTC, not SG time.  So 2020-01-19T00:04:04.443Z is 8:05am Singapore time.
# whereas Segment is stored with tiemzone at UTC.  So, could convert them all.
# TODO: But it does meant that there's a lot of events coming at the day boundary.

In [ ]:
athena_full_events_df.head(2)

In [ ]:
athena_full_events_df.dtypes

In [ ]:
segment_combined_df.dtypes

In [ ]:
# Group by columns to get around date inaccuracy issue
cols_to_group_by = ["anonymous_id", "event_name", "page_url", "date"] #, "context_ip", "context_user_agent"] #TODO: add IP address

print("Grouping by %s"% ", ".join(cols_to_group_by))

print("Fixing dates before grouping")
print("... for Segment")
segment_combined_df["date"] = segment_combined_df.apply(lambda row: row.timestamp.date().isoformat(), axis=1) # making this a string
print("... for athena")
athena_full_events_df["date"] = athena_full_events_df.apply(lambda row: row.sent_at[:10], axis=1)
# super-slow,so moving to using strings athena_full_events_df["date"] = athena_full_events_df.apply(lambda row: dateparser.parse(row.sent_at_timestamp).date(), axis=1)  #conversion from string might not be needed in the future; using dateparser as more robust, also slow

#going to reduce the number of columns to make it safer, then can go back and look for user agents etc (can do a mapping of anonymous_id to user_agent for instance.)

print("Setting sensible data types for the columns to group by")
data_type_mappings = {"event_name":"category", "date":"category"}
segment_combined_df.astype(data_type_mappings)
athena_full_events_df.astype(data_type_mappings)



In [ ]:
segment_combined_df.head()[cols_to_group_by]

In [ ]:
athena_full_events_df.head()[cols_to_group_by]

In [ ]:
# athena_full_events_df timestamp

print("Grouping by %s"%cols_to_group_by)
segment_grouped_df = segment_combined_df.groupby(cols_to_group_by).size().reset_index(name='s_count') #size preserves nulls, this sets the column to s_count

athena_grouped_df = athena_full_events_df.groupby(cols_to_group_by).size().reset_index(name='k_count')

# segment_combined_df.rename(columns = {"context_ip":"s_context_ip", "context_user_agent":"s_context_user_agent"}) 

In [ ]:
athena_grouped_df.head()

In [ ]:
# Actually join them

# set the column count names

merged_df = segment_grouped_df.merge(athena_grouped_df, how='outer', on=cols_to_group_by )

#Fill in the empty counts with 0s

merged_df["s_count"].fillna(0, inplace=True)
merged_df["k_count"].fillna(0, inplace=True)

In [ ]:
merged_df

In [ ]:
merged_df.groupby(["event_name"]).sum()

# Add Filtering Metadata

* is url blog / shop / ...
* country

In [ ]:
from urllib.parse import urlparse, parse_qs

In [ ]:
def get_metadata_from_url(url):
    p = urlparse(url.lower())
    
    #urlparse("https://www-new.moneysmart.sg/rabbit/mouse/?a=b")
    #ParseResult(scheme='https', netloc='www-new.moneysmart.sg', path='/rabbit/mouse/', params='', query='a=b', fragment='')
    
    
    nl = p.netloc
    
    page_type = ""
    stripped_path = p.path.strip("/")
    
    #blog (for SG and HK)
    if "moneysmart.tw" in nl or "moneysmart.ph" in nl or "moneysmart.id" in nl or 'blog.moneysmart' in nl or 'blog-new' in nl:
        page_type = "blog"
    
    #LPS
    elif stripped_path.endswith("ms"):
        page_type = "lps"
    
    #interstitial
    elif "iss.moneysmart" in nl or stripped_path.endswith("apply") or stripped_path.endswith("redirect"):
        page_type = "iss"
        
    #unbounce
    elif "get.moneysmart" in nl:
        page_type = "unbounce"
        
    #embed     , "regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 5) like '/embed/%' as is_embed
    
    
    #else shop
    else:
        page_type = "shop"
        
        
        
    
    #ab test side , CAST("strpos"("context"."page_url", '://www-new.') AS boolean) OR CAST("strpos"("context"."page_url", '://www3.') AS boolean)  OR CAST("strpos"("context"."page_url", '://blog3.') AS boolean) as "is_test"
    if "www-new." in nl or "www3." in nl or "blog3." in nl:
        ab_test = "test"
    else:
        ab_test = "control"
    
    
    
    slug_root = "/"+stripped_path.split("/")[0]
    
    slug = "/"+stripped_path
    
    
    """
     , CASE WHEN "regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 4) LIKE '%moneysmart.sg%' THEN 'sg' 
        WHEN "regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 4) LIKE '%moneysmart.hk%' THEN 'hk' 
        WHEN "regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 4) LIKE '%moneysmart.id%' THEN 'id' 
        WHEN "regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 4) LIKE '%moneysmart.ph%' THEN 'ph'
        WHEN "regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 4) LIKE '%moneysmart.tw%' THEN 'tw' 
        ELSE null END as country_code
    """
    
    country_code = ""
    if "moneysmart.sg" in nl:
        country_code = "sg"
    elif "moneysmart.hk" in nl:
        country_code = "hk"
    elif "moneysmart.id" in nl:
        country_code = "id"
    elif "moneysmart.ph" in nl:
        country_code = "ph"
    elif "moneysmart.tw" in nl:
        country_code = "tw"
    elif "moneysmart.com" in nl:
        country_code = "ww" #worldwide
    else:
        country_code = "??"
    
    
    #return {"page_type":page_type, "path":path, "ab_test":ab_test, "country_code":country_code}
    return [page_type, slug, slug_root, ab_test, country_code]
    
    



In [ ]:
get_metadata_from_url("https://www-new.moneysmart.sg/rabbit/headlight/?scary=True")

In [ ]:
get_metadata_from_url("https://blog.moneysmart.ph/rabbit/headlight/?scary=True")

In [ ]:
get_metadata_from_url("https://blog3.moneysmart.tw")

In [ ]:
#This is a bit slow
metadata_df = merged_df.apply(lambda x: pd.Series(get_metadata_from_url(x.page_url)), axis=1)#, index=["page_type", "path", "ab_test", "country_code"])

In [ ]:
metadata_df.rename(columns={0:"page_type", 1:"slug", 2:"slug_root", 3:"ab_test", 4:"country_code"}, inplace=True)

In [ ]:
metadata_df.head()

In [ ]:
merged_df_with_meta = pd.concat([merged_df, metadata_df], axis=1)

In [ ]:
# Set some sensible data types to speed it all up
#merged_df_with_meta.astype({"page_type":"category", "slug":"category"})
merged_df_with_meta = merged_df_with_meta.astype({"page_type":"category", "slug":"category", "ab_test":"category", "country_code":"category", "s_count":"int", "k_count":"int"})

In [415]:
merged_df_with_meta.head()

,anonymous_id,event_name,page_url,date,s_count,k_count,page_type,slug,slug_root,ab_test,country_code
0,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-18,1,1,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg
1,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://www.moneysmart.sg/embed/98e61305602380...,2020-01-18,1,1,shop,/embed/98e61305602380971d9c5e68c4a75647,/embed,control,sg
2,00000b54-600a-4de2-8700-fd9885252dca,Reading,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-18,1,1,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg
3,00000b54-600a-4de2-8700-fd9885252dca,UserView.WidgetLoad,https://www.moneysmart.sg/embed/98e61305602380...,2020-01-18,1,1,shop,/embed/98e61305602380971d9c5e68c4a75647,/embed,control,sg
4,0000b18c-192c-4f9b-b3c4-4b8f65a9e197,PageView,https://blog.moneysmart.sg/dining/yu-sheng-lo-...,2020-01-18,1,1,blog,/dining/yu-sheng-lo-hei-price-singapore,/dining,control,sg


# Top Level Checks

In [495]:
def group_by_and_show_count_difference(df, group_by_cols, with_styling=True):
    """
    This expects counts in s_count and k_count
    """
    
    grouped = df.groupby(group_by_cols).sum().reset_index()
    
    grouped["k_vs_s_%"] = grouped.apply(lambda row:(999 if row.k_count else 0) if row.s_count==0 else round(((row.k_count - row.s_count)/row.s_count)*100, 1), axis=1 )
    grouped = grouped[(grouped.k_count>0) | (grouped.s_count>0)] # filters out NaNs after grouping
    
    

    return grouped

In [503]:
def colour_grouped_table(df):

        

    def color_how_good(value):
        av = abs(value)
        if av<2:
            c = "green"
        elif value <0:
            c =  "red"

        else:
            c= "blue"

        if av>20:
            return "background-color:rgb(250,200,200)"
        return "color:%s"%c # it's just CSS, so you can do background as well.
    return df.style.applymap(color_how_good , subset=["k_vs_s_%"])


In [504]:
merged_df_with_meta.head()

,anonymous_id,event_name,page_url,date,s_count,k_count,page_type,slug,slug_root,ab_test,country_code
0,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-18,1,1,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg
1,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://www.moneysmart.sg/embed/98e61305602380...,2020-01-18,1,1,shop,/embed/98e61305602380971d9c5e68c4a75647,/embed,control,sg
2,00000b54-600a-4de2-8700-fd9885252dca,Reading,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-18,1,1,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg
3,00000b54-600a-4de2-8700-fd9885252dca,UserView.WidgetLoad,https://www.moneysmart.sg/embed/98e61305602380...,2020-01-18,1,1,shop,/embed/98e61305602380971d9c5e68c4a75647,/embed,control,sg
4,0000b18c-192c-4f9b-b3c4-4b8f65a9e197,PageView,https://blog.moneysmart.sg/dining/yu-sheng-lo-...,2020-01-18,1,1,blog,/dining/yu-sheng-lo-hei-price-singapore,/dining,control,sg


## By Country


In [505]:
g = group_by_and_show_count_difference(merged_df_with_meta, ["country_code"])

In [506]:
colour_grouped_table(g)

,country_code,s_count,k_count,k_vs_s_%
0,??,297,22,-92.6
1,hk,0,71925,999
2,id,0,133,999
3,ph,0,3610,999
4,sg,301276,312253,3.6
5,tw,0,50846,999


## By Event Type

In [507]:
g = group_by_and_show_count_difference(merged_df_with_meta, ["country_code", "event_name"])
g.sort_values(["country_code", "event_name"], inplace=True)
colour_grouped_table(g)

,country_code,event_name,s_count,k_count,k_vs_s_%
11,??,PageView,297,12,-96
12,??,Reading,0,10,999
29,hk,LeadGeneration.ClickConversion,0,97,999
30,hk,LeadGeneration.ClickedApplyButton,0,127,999
31,hk,LeadGeneration.ClickedCTA,0,398,999
32,hk,LeadGeneration.Conversion,0,640,999
34,hk,LeadGeneration.FormSubmitted,0,100,999
36,hk,LeadGeneration.RedirectCompleted,0,620,999
37,hk,LeadGeneration.ShowedMoreDetails,0,1588,999
38,hk,LeadGeneration.ViewedMoreDetails,0,740,999


## By Top Level Slug

In [494]:
g = group_by_and_show_count_difference(merged_df_with_meta[merged_df_with_meta.page_type!="blog"], ["country_code", "slug_root", "event_name"])
g.sort(["country_code", "slug_root", "event_name"])
g

,country_code,slug_root,s_count,k_count,k_vs_s_%
7,??,/c:,8,10,25
67,??,/search,0,1,999
73,??,/translate_c,0,11,999
80,??,/www.moneysmart.sg,289,0,-100
82,hk,/,0,14,999
103,hk,/embed,0,8264,999
104,hk,/en,0,1669,999
163,hk,/zh-hk,0,14202,999
328,sg,/,872,1194,36.9
329,sg,/.,12,0,-100


## By Type of Page

In [480]:
group_by_and_show_count_difference(merged_df_with_meta, ["country_code", "page_type"])

  country_code page_type  s_count  k_count
0           ??      blog      NaN      NaN
1           ??       iss      NaN      NaN


,country_code,page_type,s_count,k_count,k_vs_s_%
0,??,blog,nan,nan,nan
1,??,iss,nan,nan,nan
2,??,lps,nan,nan,nan
3,??,shop,297,22,-92.6
4,??,unbounce,nan,nan,nan
5,hk,blog,0,47776,999
6,hk,iss,0,2368,999
7,hk,lps,0,751,999
8,hk,shop,0,21030,999
9,hk,unbounce,nan,nan,nan


# Other Issues to Check For
* duplicates
* skipping "https" from the url (observed as a current issue)
* certain browsers having issues

In [407]:
round(0.12312, 2)

0.12

In [455]:
import numpy as np

np.random.seed(24)
df = pd.DataFrame({'A': np.linspace(1, 10, 10)})
df = pd.concat([df, pd.DataFrame(np.random.randn(10, 4), columns=list('BCDE'))],
               axis=1)
df.iloc[0, 2] = np.nan

In [456]:

def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color

In [457]:
s = df.style.applymap(color_negative_red)
s

,A,B,C,D,E
0,1,1.32921,nan,-0.31628,-0.99081
1,2,-1.07082,-1.43871,0.564417,0.295722
2,3,-1.6264,0.219565,0.678805,1.88927
3,4,0.961538,0.104011,-0.481165,0.850229
4,5,1.45342,1.05774,0.165562,0.515018
5,6,-1.33694,0.562861,1.39285,-0.063328
6,7,0.121668,1.2076,-0.00204021,1.6278
7,8,0.354493,1.03753,-0.385684,0.519818
8,9,1.68658,-1.32596,1.42898,-2.08935
9,10,-0.12982,0.631523,-0.586538,0.29072
